Automatische FEA für eine Geometrieanalyse:

Ziel: Filterung der Materialien

Annahmen:   
- volles Volumen, 
- schwächstes Modul in alle Richtungen annehmen für Isotropes Material

- Druck, oder Strecktest simulation in alle sechs Raumrichtungen 

- Von Mises Yield criterion für plastische Einschätzung


Prozessablauf: 
1. Der Kunde erstellt seinen Auftrag und wir bekommen das Object
2. Der Kunde spezifiziert die Anforderungen an das Bauteil 
3. Falls Druck nötig ist, wird geprüft, welche Materialien eine größere Yield spannung haben 
4. Geometrieanalyse, um zu schauen, ob auch das Object den Druck stanhält. 


            

In [22]:
import numpy as np
from skfem import *
import pytetwild
import meshio
from skfem.models.elasticity import (linear_elasticity, lame_parameters, linear_stress)
from skfem.helpers import dot, sym_grad
from skfem.visuals.matplotlib import draw, plot
import  matplotlib.pyplot as plt



materials = {
    "PLA": {
        "Youngs Modulus": 2636e6,  # Pa (2636 MPa)
        "Poisson Ratio": 0.327,
        "Yielding Stress": 54.4e6,  # Pa (54.4 MPa)
    },
    "ABS": {
        "Youngs Modulus": 2400e6,  # Pa (2400 MPa)
        "Poisson Ratio": 0.37,
        "Yielding Stress": 26.84e6  # Pa (26.84 MPa)
    }
}
def visualize(m_shifted, vonmises, dgbh):
    shiftedbase = Basis(m_shifted,dgbh)
    ax = plot(shiftedbase,vonmises, shading='gouraud',colorbar =r"$\sigma_{\mathrm{mises}}$")
    draw(m_shifted, ax = ax)
    return ax
def yield_stress(array, x):
    e = []
    p = []
    for i in range(len(array)):
        if array[i] > x:
            p.append(i)
        
    return {'plastic': p}

def FEM_PressTest(material, pressure, object):
    """material nimmt als input String"""
    """pressure nimmt als input int"""
    """object nimmt als input .STL file"""
    
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
       # Define FE mesh elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)
    
    young_modulus = materials[material]["Youngs Modulus"]
    poisson_ratio = materials[material]["Poisson Ratio"]
    lame_params = lame_parameters(young_modulus, poisson_ratio)
    
    K = asm(linear_elasticity(*lame_params), ib)
    
        # Define boundary conditions based on the test type (tensile or compression)
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }
    
    displacements = {
        'left': ('right', 'u^1', pressure),
        'right': ('left', 'u^1', -pressure),
        'up': ('down', 'u^2', -pressure),
        'down': ('up', 'u^2', pressure),
        'front': ('back', 'u^3', -pressure),
        'back': ('front', 'u^3', pressure)
    }
    
    
    
    
    results = {}
    for direction, (opposite, component, press_value) in displacements.items():
        u = ib.zeros()

        # Initialize force array
        F = np.zeros(u.shape)

        # Correctly index the nodal DOFs
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = press_value

        # Fix boundary conditions
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0

        # Set DOFs and solve the system
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        K_free = K[free_dofs][:, free_dofs]
        F_free = F[free_dofs]
        u_free = solve(K_free, F_free)
        u[free_dofs] = u_free
        # compute the strain tensor
         # Calculate final shift
        final_shift = u[direction_dofs]

        # Implement shifts
        u[direction_dofs] = final_shift
        I = ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))
        u = solve(*condense(K, x=u, I=I))
        
        
        
        sf = 1000
        
        
        # Translate and save the mesh
        m_shifted = mesh.translated(sf * u[ib.nodal_dofs])
        # draw(m_shifted) 
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/Compression_TESTS_{object}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
        
        #vonmises calculation 
        s = {}
        dgb = ib.with_element(ElementTetP0())
        
        
        up = ib.interpolate(u)

        C = linear_stress(*lame_params)
        
        
        for i in [0, 1]:
            for j in [0, 1]:
                s[i, j] = dgb.project(C(sym_grad(up))[i, j])

        s[2, 2] = poisson_ratio * (s[0, 0] + s[1, 1])

        vonmises = np.sqrt(.5 * ((s[0, 0] - s[1, 1]) ** 2 +
                                (s[1, 1] - s[2, 2]) ** 2 +
                                (s[2, 2] - s[0, 0]) ** 2 +
                                6. * s[0, 1] ** 2))
        material_yield_stress = materials[material]['Yielding Stress']
        
        
        
        yield_results = yield_stress(vonmises, material_yield_stress)
        results[direction] = yield_results
    return results
        
    



def FEM_TensileTest(material, pressure, object):
       # Read the STL file and create the tetrahedral mesh
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    cells = meshio.CellBlock('tetra', tetrahedras)
    p = np.array([vertices[:, 0], vertices[:, 1], vertices[:, 2]], dtype=np.float64)
    t = np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
       # Define FE mesh elements
    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)
    
    young_modulus = materials[material]["Youngs Modulus"]
    poisson_ratio = materials[material]["Poisson Ratio"]
    lame_params = lame_parameters(young_modulus, poisson_ratio)
    
    K = asm(linear_elasticity(*lame_params), ib)
    
        # Define boundary conditions based on the test type (tensile or compression)
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }
    
    displacements = {
        'left': ('right', 'u^1', -pressure),
        'right': ('left', 'u^1', pressure),
        'up': ('down', 'u^2', pressure),
        'down': ('up', 'u^2', -pressure),
        'front': ('back', 'u^3', pressure),
        'back': ('front', 'u^3', -pressure)
    }
    
    
    
    
    results = {}
    for direction, (opposite, component, press_value) in displacements.items():
        u = ib.zeros()

        # Initialize force array
        F = np.zeros(u.shape)

        # Correctly index the nodal DOFs
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = press_value

        # Fix boundary conditions
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0

        # Set DOFs and solve the system
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        K_free = K[free_dofs][:, free_dofs]
        F_free = F[free_dofs]
        u_free = solve(K_free, F_free)
        u[free_dofs] = u_free
        # compute the strain tensor
         # Calculate final shift
        final_shift = u[direction_dofs]

        # Implement shifts
        u[direction_dofs] = final_shift
        I = ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))
        u = solve(*condense(K, x=u, I=I))
        
        
        
        sf = 1000
        
        
        # Translate and save the mesh
        m_shifted = mesh.translated(sf * u[ib.nodal_dofs])
        
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/TensileTESTS_{object}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
        m_shifted = MeshTet()
        #vonmises calculation 
        s = {}
        dgb = ib.with_element(ElementTetP1())
        
        
        up = ib.interpolate(u)

        C = linear_stress(*lame_params)
        
        
        for i in [0, 1]:
            for j in [0, 1]:
                s[i, j] = dgb.project(C(sym_grad(up))[i, j])

        s[2, 2] = poisson_ratio * (s[0, 0] + s[1, 1])

        vonmises = np.sqrt(.5 * ((s[0, 0] - s[1, 1]) ** 2 +
                                (s[1, 1] - s[2, 2]) ** 2 +
                                (s[2, 2] - s[0, 0]) ** 2 +
                                6. * s[0, 1] ** 2))
        material_yield_stress = materials[material]['Yielding Stress']
        
        
        # visualize(m_shifted, vonmises, dgb)
        yield_results = yield_stress(vonmises, material_yield_stress)
        
        
        results[direction] = yield_results
        
    return results

result = FEM_TensileTest("PLA", 1e8, "Cube_3d_printing_sample (1).stl")    
print(result)


Starting tetrahedralization process...


Transforming over 1000 elements to C_CONTIGUOUS.


Tetrahedralization complete.
Number of vertices: 2167
Number of tetrahedra: 9746
Prepared numpy array for points.
Prepared numpy array for tetrahedra.
Tetrahedralization process completed successfully.
2167
2167
2167
2167
2167
2167
{'left': {'plastic': [0, 1, 2, 3, 4, 5, 14, 74, 75, 76, 84, 101, 143, 603, 615, 616, 624, 632, 647, 671, 672, 678, 679, 776, 777, 778, 966, 974, 990, 1006, 1030, 1062, 1207, 1244, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1320, 1324, 1331, 1333, 1334, 1342, 1353, 1355, 1356, 1361, 1367, 1371, 1374, 1413, 1414, 1423, 1424, 1426, 1428, 1429, 1489, 1490, 1493, 1497, 1499, 1500, 1504, 1505, 1527, 1528, 1529, 1530, 1531, 1532, 1533, 1534, 1568, 1571, 1581, 1619, 1621, 1706, 1735, 1738, 1741, 1758, 1759, 1770, 1776, 1868, 1871, 1906, 1922, 1928, 1935, 1938, 1944, 1995, 1996, 1998, 2011, 2016, 2017, 2025, 2054, 2056, 2066, 2071, 2073, 2079, 2099, 2104, 2105, 2112, 2113, 2117, 2119, 2121, 2125, 2144, 2154, 2166]}, 'right': {'plastic': [2, 3, 4, 5, 6, 7, 20, 75

In [15]:
import numpy as np
from skfem import *
import pytetwild
import meshio
from skfem.models.elasticity import linear_elasticity, lame_parameters, linear_stress
from skfem.helpers import dot, sym_grad
from skfem.visuals.matplotlib import draw, plot
import matplotlib.pyplot as plt
from tvtk.api import tvtk
from tvtk.common import configure_input_data

def save_vtk(m_shifted, vonmises, save_path):
    # Erstellen eines TVTK UnstructuredGrid Objekts
    points = m_shifted.p.T
    cells = m_shifted.t.T
    tetra_cells = np.hstack([np.full((cells.shape[0], 1), 4), cells])
    
    grid = tvtk.UnstructuredGrid(points=points)
    grid.set_cells(tvtk.Tetra().cell_type, tetra_cells)
    
    # Hinzufügen der Von Mises Spannungen als Skalare
    grid.point_data.scalars = vonmises
    grid.point_data.scalars.name = 'Von Mises Stress'
    
    # Speichern als VTK-Datei
    writer = tvtk.XMLUnstructuredGridWriter(file_name=save_path)
    writer.set_input_data(grid)
    writer.write()

# Materialeigenschaften
materials = {
    "PLA": {"Youngs Modulus": 2636e6, "Poisson Ratio": 0.327, "Yielding Stress": 54.4e6},
    "PLA_Chacon": {"Youngs Modulus":3765e6, "Poisson Ratio":0.35, "Yielding Stress":20.2e6}, #Werte Entnommen aus Chacon et al. 2017
    "ABS": {"Youngs Modulus": 2400e6, "Poisson Ratio": 0.37, "Yielding Stress": 26.84e6}
}

def yield_stress(array, threshold):
    return {'plastic': [i for i in range(len(array)) if array[i] > threshold]}

def run_FEM_test(material, pressure, object_file, test_type):
    # Einlesen der STL-Datei
    stl = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{object_file}")
    vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])
    p, t = np.array(vertices.T, dtype=np.float64), np.array(tetrahedras, dtype=np.float64).T
    mesh = MeshTet(p, t)
    
    e1, e = ElementTetP1(), ElementVector(ElementTetP1())
    ib = Basis(mesh, e, MappingIsoparametric(mesh, e1), 3)
    print(ib)
    
    # Materialparameter
    young_modulus, poisson_ratio = materials[material]["Youngs Modulus"], materials[material]["Poisson Ratio"]
    lame_params = lame_parameters(young_modulus, poisson_ratio)
    K = asm(linear_elasticity(*lame_params), ib)
    
    # Freiheitsgrade (DOFs) definieren
    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }
    
    # Verschiebungen definieren
    displacements = {
        'left': ('right', 'u^1', pressure if test_type == 'compression' else -pressure),
        'right': ('left', 'u^1', -pressure if test_type == 'compression' else pressure),
        'up': ('down', 'u^2', -pressure if test_type == 'compression' else pressure),
        'down': ('up', 'u^2', pressure if test_type == 'compression' else -pressure),
        'front': ('back', 'u^3', -pressure if test_type == 'compression' else pressure),
        'back': ('front', 'u^3', pressure if test_type == 'compression' else -pressure)
    }
    
    results = {}
    for direction, (opposite, component, press_value) in displacements.items():
        u, F = ib.zeros(), np.zeros(ib.zeros().shape)
        direction_dofs = dofs[direction].nodal[component].astype(int).flatten()
        F[direction_dofs] = press_value
        fixed_dofs = np.hstack([dofs[opposite].all()])
        u[fixed_dofs] = 0
        free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
        u[free_dofs] = solve(K[free_dofs][:, free_dofs], F[free_dofs])
        u = solve(*condense(K, x=u, I=ib.complement_dofs(np.concatenate([dofs[direction], dofs[opposite]]))))
        m_shifted = mesh.translated(1 * u[ib.nodal_dofs])
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/{test_type}TESTS_{object_file}_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})
       
        s, dgb = {}, ib.with_element(ElementTetP0())
        up = ib.interpolate(u)
        C = linear_stress(*lame_params)
        for i in [0, 1]:
            for j in [0, 1]:
                s[i, j] = dgb.project(C(sym_grad(up))[i, j])
        s[2, 2] = poisson_ratio * (s[0, 0] + s[1, 1])
        vonmises = np.sqrt(.5 * ((s[0, 0] - s[1, 1]) ** 2 + (s[1, 1] - s[2, 2]) ** 2 + (s[2, 2] - s[0, 0]) ** 2 + 6. * s[0, 1] ** 2))
        
        print(len(vonmises), vonmises)
        save_path = f"C:/Users/AlexP/Desktop/BA/Meshes/Solutions/Vonmises_{object_file}_{direction}.vtk"
        save_vtk(m_shifted, vonmises, save_path)
        results[direction] = yield_stress(vonmises, materials[material]['Yielding Stress'])
    return results

# Beispielaufruf der Funktion
result = run_FEM_test("PLA_Chacon", 1, "TestobjektStreckversuch.stl", 'tensile')
print(result)

Starting tetrahedralization process...
Tetrahedralization complete.
Number of vertices: 310
Number of tetrahedra: 928
Prepared numpy array for points.
Prepared numpy array for tetrahedra.
Tetrahedralization process completed successfully.
<skfem CellBasis(MeshTet1, ElementVector) object>
  Number of elements: 928
  Number of DOFs: 930
  Size: 5345280 B
928 [0.03179276 0.00108568 0.0218578  0.02463641 0.02396246 0.00137067
 0.00116142 0.00112762 0.00264193 0.00616446 0.00280663 0.02048
 0.03014794 0.00167584 0.01194438 0.00158072 0.00089822 0.01141089
 0.00138368 0.00127155 0.00107764 0.00291352 0.00100132 0.02894237
 0.02555214 0.00094896 0.01874962 0.00153258 0.00291392 0.00034162
 0.00114349 0.00775007 0.00783636 0.00119281 0.00199389 0.0136794
 0.03827258 0.02476256 0.01860615 0.00043837 0.00150283 0.02557484
 0.00091564 0.01003202 0.00089879 0.00149613 0.00126801 0.0012331
 0.00155318 0.01067593 0.00216272 0.01146743 0.02464558 0.02032954
 0.02292552 0.02700786 0.02482603 0.0203552